In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
#2
df = pd.read_csv('drug200.csv')

In [4]:
#3 kiem tra du lieu dung ham info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [5]:
#4 mo ta du lieu dung ham describe
df.describe()

,Age,Na_to_K
count,200.000000,200.000000
mean,44.315000,16.084485
std,16.544315,7.223956
min,15.000000,6.269000
25%,31.000000,10.445500
50%,45.000000,13.936500
75%,58.000000,19.380000
max,74.000000,38.247000


In [ ]:
#5.  Kiểm tra kiểu dữ liệu, dùng types
#6.  Kiểm tra tên cột, dùng columns
#7.  Kiểm tra dữ liệu trống, dùng df.isnull().sum()
#8.  Kiểm tra dữ liệu trùng, dùng df.duplicated().sum()

In [7]:
#9 doi ten cot phu hop
df.rename(columns= {'Na_to_K' : 'Sodium_to_Potassium', 'BP' : 'Blood_Pressure'}, inplace=True)
df['Sex'].replace({'M':'Male', 'F': 'Female'},inplace=True)
df['Sodium_to_Potassium'] = df ['Sodium_to_Potassium'].round(0)
df['Sodium_to_Potassium'] = df ['Sodium_to_Potassium'].astype(int)
df

,Age,Sex,Blood_Pressure,Cholesterol,Sodium_to_Potassium,Drug
0,23,Female,HIGH,HIGH,25,drugY
1,47,Male,LOW,HIGH,13,drugC
2,47,Male,LOW,HIGH,10,drugC
3,28,Female,NORMAL,HIGH,8,drugX
4,61,Female,LOW,HIGH,18,drugY
...,...,...,...,...,...,...
195,56,Female,LOW,HIGH,12,drugC
196,16,Male,LOW,HIGH,12,drugC
197,52,Male,NORMAL,HIGH,10,drugX
198,23,Male,NORMAL,NORMAL,14,drugX


In [8]:
#10 thong ke so luong nguoi thuoc 5 nhom tuoi cao nhat
age_values = df['Age'].value_counts()
top_age = age_values.head(5)
df_top_age = pd.DataFrame({'Age': top_age.index, 'Count' : top_age.values})
df_top_age

,Age,Count
0,47,8
1,49,7
2,23,7
3,28,7
4,39,6


In [9]:
#11 tien hanh EDA du lieu

# EDA

def create_plot(ax, x, data, plot_type='count', y=None, palette='Set2'):

    if plot_type == 'count':

        sns.countplot(x=x, data=data, palette-palette, ax=ax)

    elif plot_type == "bar":

        sns.barplot(x=x, y=y, data=data, palette-palette, ax=ax)

    ax.set_title(f'Plot of {x}' if plot_type = 'count' else f'Bar Plot of {x} and {y}')

    for p in ax.patches:

        ax.annotate(f'{int(p.get_height())}',

                    (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='baseline",fontsize=10, color='black',

                    xytext=(0, 1),textcoords="offset points')

# Create the figure and subplots

plt.figure(figsize=(10, 10))

# Define plot configurations

plot_configs = [

{'x': 'Sex', 'data': df, 'plot_type': 'count'},

{'x': 'Blood_Pressure', 'data': df, 'plot_type': 'count'},

{'x': 'Cholesterol', 'data': df, 'plot_type': 'count'},

{'x': 'Drug', 'data': df, 'plot_type': 'count'},

{'x': 'Age', 'y": "Count', 'data': df_top_age, 'plot_type': 'bar'}

]

# Loop through plot configurations to create subplots

for i, config in enumerate (plot_configs):

ax = plt.subplot(3, 3, i+1)

create_plot(ax, **config)

plt.tight_layout()

plt.show()

IndentationError: expected an indented block (<ipython-input-9-699f0331d72c>, line 7)